__Definitions:__
<br>

|Variable | Meaning                                                 |
|---------|---------------------------------------------------------|
|$W$:     | susceptible and wise                                    |
|$R$:     | susceptible and risky                                   |
|$I$:     | infectious                                              |
|$D$:     | dead                                                    |
|$S$:     | individuals recovered and who can no longer be infected |

__Reactions:__
<br>
$$
\begin{align}
    W &\rightarrow^a R &&\\
    R &\rightarrow^{\alpha} W&&\\
    R + I &\rightarrow^c I + I&&\\
    W + I &\rightarrow^b I + I&&\\
    R + I &\rightarrow^f W + I &&\text{(if a risky encounters a (known) infected, it gets wise! - David's idea)}\\
    I &\rightarrow^{\beta} W &&\text{(if you were infected and then cured, you get wise!😁)}\\
    I &\rightarrow^d D&&\\
    I &\rightarrow^{\rho} S&&
\end{align}
$$

In [1]:
import numpy as np

import matplotlib.pyplot as plt

import ipywidgets as wgs
from ipywidgets import interact
from IPython.display import display

In [2]:
## Input parameters ####################
#N =  200   # total population
#t =    0.0 # start time
#T = 1000.0 # maximum elapsed time
#
## Initial conditions on Infectious, Dead, Saved, Risky and Wise people 
#n_I = 1
#n_D = 0
#n_S = 0
#n_R = 100
#n_W = N - n_I - n_R
#
## Rates
#WtoR   = 0.5    # rate of wise to risky
#RtoW   = 0.03   # rate of risky to wise
#RItoII = 0.01   # rate of risky to infected
#WItoII = 0.001  # rate of wise to infected
#RItoWI = 0.001  # rate of risky gets wise by encountering infected
#ItoW   = 0.2    # rate of infected to cured (and wise)
#ItoD   = 0.08   # rate of infected to deceased 
#ItoS   = 0.1    # rate of infected to recovered (and immune)
#
## For algorithm
#seed = 42

In [98]:
def GillespieSIRwr(N = 200, n_R = 100,
                   WtoR = 0.5, RtoW = 0.03, RItoII = 0.01, WItoII = 0.001, RItoWI = 0.000, ItoW = 0.2, ItoD = 0.08, ItoS = 0.1,
                   rdt = 0.,
                   seed = 42):
    # Inital values (non-interactive)
    t   =    0.0
    T   = 1000.0
    rt  = t
    
    # Initial conditions on Infectious, Dead, Saved, Risky and Wise people 
    n_I  = 1
    n_D  = 0
    n_S  = 0
    n_rI = 0 # Reporting "misses" the first infected
    n_W  = N - n_I - n_R

    # Vectors for plotting
    time  = []
    rtime = [rt] # Time scale for reporting number of infected
    nR    = []
    nI    = []
    nrI   = [n_rI] # Number of reported infected
    nW    = []
    nD    = []
    nS    = []

    # Random Generator
    rg = np.random.RandomState(seed)

    # Main loop
    while t < T:
        if n_I == 0:
            break
        # Rates calculation
        w1  = WtoR/(n_rI+1) * n_W # Modified reaction taking account of the varietion of infection n_I
        w2  = RtoW         * n_R
        w3  = RItoII       * n_I * n_R
        w4  = WItoII       * n_I * n_W
        w5  = RItoWI       * n_I * n_R
        w6  = ItoW         * n_I
        w7  = ItoD         * n_I
        w8  = ItoS         * n_I

        W   = w1 + w2 + w3 + w4 + w5 + w6 + w7 + w8
        # First uniformily distributed ranmdom number
        r1  = rg.uniform(0.0, 1.0)
        # Time increment
        tau = -np.log(r1) / W
        t   = t + tau

        
        # Second uniformily distributed ranmdom number
        r2 = rg.uniform(0.0, 1.0)
        # Choice of the reaction according to the probabilities
        if r2 < w1 / W:
            n_W = n_W - 1
            n_R = n_R + 1
        if r2>=w1/W and r2 < (w1+w2) / W:
            n_R = n_R - 1
            n_W = n_W + 1
        if r2>=(w1+w2)/W and r2 < (w1+w2+w3)/W:
            n_R = n_R - 1
            n_I = n_I + 1
        if r2>=(w1+w2+w3)/W and r2 < (w1+w2+w3+w4)/W:
            n_W = n_W - 1
            n_I = n_I + 1
        if r2>=(w1+w2+w3+w4)/W and r2 < (w1+w2+w3+w4+w5)/W:
            n_R = n_R - 1
            n_W = n_W + 1
        if r2>=(w1+w2+w3+w4+w5)/W and r2 < (w1+w2+w3+w4+w5+w6)/W:
            n_I = n_I - 1
            n_W = n_W + 1
        if r2>=(w1+w2+w3+w4+w5+w6)/W and r2< (w1+w2+w3+w4+w5+w6+w7)/W:
            n_I = n_I - 1
            n_D = n_D + 1
        if r2 >(w1+w2+w3+w4+w5+w6+w7)/W:
            n_I = n_I - 1
            n_S = n_S + 1
            
        # Update report of number of infected
        if rdt != 0. and t//rdt != rt:
            rt = t//rdt
            n_rI = n_I
            rtime.append(t)
            nrI.append(n_rI)
        if rdt == 0.:
            rt = t
            n_rI = n_I
            rtime.append(t)
            nrI.append(n_rI)
        
        # Construction of vectors for plotting   
        time.append(t)
        nW.append(n_W)
        nI.append(n_I)
        nR.append(n_R)
        nS.append(n_S)
        nD.append(n_D)

    # Plotting
    fig, ax = plt.subplots(figsize=(20,10))

    line1, = ax.step(time,  nW,  where='post', linewidth=3, color="tab:green",  zorder=3, label=r'nW$_{ise}$')
    line2, = ax.step(time,  nI,  where='post', linewidth=3, color="tab:orange", zorder=2, label=r'nI$_{nfectious}$')
    line3, = ax.step(time,  nR,  where='post', linewidth=3, color="tab:red",    zorder=3, label=r'nR$_{isky}$')
    line4, = ax.step(time,  nS,  where='post', linewidth=2, color="tab:blue",   zorder=1, label=r'nS$_{aved}$', alpha=0.2)
    line5, = ax.step(time,  nD,  where='post', linewidth=2, color="black",      zorder=1, label=r'nD$_{ead}$',  alpha=0.2)
    line6, = ax.step(rtime, nrI, ':', where='post', linewidth=2, color="tab:orange", zorder=3, label=r'nr$_{reported}$I$_{nfectious}$')

    legend = ax.legend(loc='best', shadow=True, fontsize='x-large')
    plt.title('Simulation of SIR_wr model')
    plt.xlabel('Time')
    plt.ylabel('Individuals')
    #plt.ylim((0,N))
    
    plt.show();

## Sliders etc.

In [99]:
sN      = wgs.IntSlider(min=0, max=1000, step=10, value=200, continuous_update=False, description=r"N :")
sn_R    = wgs.IntSlider(min=0, max=1000, step=10, value=100, continuous_update=False, description=r"n_R :")

sWtoR   = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.500, continuous_update=False, readout_format='.3f', description=r"W   $\rightarrow$ R  :")
sRtoW   = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.030, continuous_update=False, readout_format='.3f', description=r"R   $\rightarrow$ W  :")
sRItoII = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.010, continuous_update=False, readout_format='.3f', description=r"R+I $\rightarrow$ I+I:")
sWItoII = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.001, continuous_update=False, readout_format='.3f', description=r"W+I $\rightarrow$ I+I:")
sRItoWI = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.000, continuous_update=False, readout_format='.3f', description=r"R+I $\rightarrow$ W+I:")
sItoW   = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.200, continuous_update=False, readout_format='.3f', description=r"I   $\rightarrow$ W  :")
sItoD   = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.080, continuous_update=False, readout_format='.3f', description=r"I   $\rightarrow$ D  :")
sItoS   = wgs.FloatSlider(min=0., max=1., step=0.001, value=0.100, continuous_update=False, readout_format='.3f', description=r"I   $\rightarrow$ S  :")

srdt    = wgs.FloatSlider(min=0., max=10., step=1.0,  value=0.000, continuous_update=False, readout_format='.1f', description=r"rdt :")


sseed   = wgs.IntSlider(min=0, max=100, step=1, value=42, continuous_update=False, description="seed")

uiBasic = wgs.VBox([wgs.Label(r"Basic Settings:"), sN, sn_R, wgs.Label(""), wgs.Label(r"Reporting interval:"), srdt, wgs.Label(""), wgs.Label(r"Algorithm:"), sseed]) 
uiRates = wgs.VBox([wgs.Label(r"Rates:"), sWtoR, sRtoW, sRItoII, sWItoII, sRItoWI, sItoW, sItoD, sItoS]) 
ui      = wgs.HBox([uiBasic, uiRates])

controls ={"N": sN, "n_R": sn_R,
           "WtoR": sWtoR, "RtoW": sRtoW, "RItoII": sRItoII, "WItoII": sWItoII, "RItoWI": sRItoWI, "ItoW": sItoW, "ItoD": sItoD, "ItoS": sItoS,
           "rdt": srdt, 
           "seed": sseed}

out = wgs.interactive_output(GillespieSIRwr, controls)
display(ui, out)

Output()